# Tutorial for Google Cloud Platform

We will talk about how to setup your _Google Cloud Platform_ (GCP) project with the education grant coupons you have. Please read the instructions carefully.

1. We will create a GCP project owned by both team members that will be paid by your coupons.
2. We will create a _virtual machine_ (VM) with GPU, that can be used for training neural networks.
3. We will discuss a training workflow for this VM.

## Preliminaries

Both team members should have their STU G Suite account activated, i.e. you should be able to log in to your Google account (e.g. in Gmail) via your `stuba.sk` email address. You only need AIS credentials to activate this account.

## Creating GCP Project

1. You have received a invitation link. Open this link and fill out the form. Use your `stuba.sk` email address, e.g. `xsurname@stuba.sk`. [(Image reference)](../backstage/gcp_tutorial_images/grants.png)

2. You will receive a verification email. This email can be found in your [`webmail.stuba.sk`](https://webmail.stuba.sk/) account. Click on the link in this email to verify your address.

3. You will receive an email with coupons. Next step is different for each member of the team:

    __First member__

    3.1 Click the link in the email with the coupon. You will see the _Education grants_ page [(Image reference)](../backstage/gcp_tutorial_images/redeem.png). First, make sure that you are logged in in your `stuba.sk` account (top right). If you are not, switch the accounts.   
    3.2 Fill out your coupon code and other form fields, then click _Accept and continue_.  
    3.3 You should now see a GCP Console dashboard  [(Image reference)](../backstage/gcp_tutorial_images/dashboard.png). This is where you can manage your GCP projects.      
    3.4 You can check if your coupon went through. Navigate to _☰ > Billing_. You should see that you have a _Neural Networks_ billing account with 50$ credit.  [(Image reference)](../backstage/gcp_tutorial_images/billing.png)
    3.5 Add your teammate to your project. Go back to the dashboard (click on the GCP logo) and use the option _Add people to this project_ in _Project info_ section. Fill your teammate's `stuba.sk` email address and pick _Project > Owner_ as a role.  
    3.6 You can also use _Go to project settings_ option from the dashboard to change your project's name.
    
    __Second member__
    
    3.1 __Do NOT open the link in the email.__ Instead, send your coupon code to your teammate. He should open https://console.cloud.google.com/edu. There he needs to make sure that he is logged in in his `stuba.sk` account. After he uses the coupon you send him, your project should now have 100$ credit available.

After you finish the steps above, both of you should see your project in your GCP interface. You select your projects in the top left dropdown menu right beside the GCP logo. The first team member should also see that your team has a 100$ credit available in the _Billing_ section.

## Creating Virtual Machine

### GPU Quotas

By default we have 0 GPUs available for our projects. We need to send a quota request to get some GPUs:

1. Navigate to _☰ > IAM & admin > Quotas_.
2. Use _Metric_ filter to first unselect all the metrics _(None)_ and then use the text search field to find _GPUs (all regions)_. [(Image reference)](../backstage/gcp_tutorial_images/quotas.png)
3. Select this metric viac checkbox and click _Edit quotas_ option.
4. Then you need to fill your contact option in the popup window on the right and finally ask for quota increase.
5. You can ask for _4_ GPUs (you will usually use just one, but it does not hurt to have a reserve) and as _Request description_ write that you need it for "Deep learning university course".
6. Google should process your request in two workdays. My experience is that it usually takes only a few hours. You will be notified via email. Then you can continue with this tutorial.

### Creating VM

After you receive a notification that your quota was increased, you can create your very own VM with GPU.

1. Navigate to _☰ > Compute Engine > VM Instances_
2. Select _Create instance_
3. Pick your options:
    - Select the _Machine Type_ as _Custom_ and select the number of CPU cores and RAM. These should be set according to your project requirements. I would recommend to start with 2 cores and 12 GB RAM.
    - After you click on _CPU platform and GPU_ you can select the number of GPUs. First, I would recommend using only 1 GPU. The cheapest are K80s (~0.35$/h). Different regions provide different GPUs. Pick the region (at the top of the form) accordingly to this [table](https://cloud.google.com/compute/docs/gpus/#gpus-list). I would recommend using European regions.
    - Select a boot image. You can use your own Docker image. Otherwise I would suggest using _Ubuntu 16.04 LTS,_ which is also used for the TensorFlow images. You can start with a 30GB SSD disk (you need ~7GB for system, some space for you data and logs from your training). If you have a big training set, set the HDD size accordingly. 
    - Finally select the _Management, security, disks, networking, sole tenancy_ option and set _Automatic Restart_ to _Off_. More on this later. [(Image reference)](../backstage/gcp_tutorial_images/vm_setup.png)
4. Create your VM

You can see the hourly estimate top right, it should be ~0.50$/h. This does not include tax. You can change all the options (RAM size, number of CPUs, etc) later, so don't stress too much about them.

### Managing VM

After you create your VM, you can manage it from _☰ > Compute Engine > VM Instances_ menu. Three vertical dots at the right should give you the option to start or stop a VM. While it has the green tick icon, the VM is running.

<p style='color: red; font-weight: bold'>Make sure your turn off your VM when you are not using it.</p>

It is billed for each minute it runs. If you leave it running without training, you are wasting your credit. You can monitor your CPU usage from the main page dashboard, or you can even use a _Cloud Console_ mobile app to monitor it from your phone. CPU usage can tell you if a script is running on the VM or if it is idle.  [(Image reference)](../backstage/gcp_tutorial_images/vms.png)

### Connecting to Your VM

You can connect to your VM via _Cloud SDK_. Cloud SDK is a console based tool that lets you use commands such as `ssh` or `scp` and connect to your VM. Install Cloud SDK using the [official installation guide](https://cloud.google.com/sdk/install), it has different versions for different operations systems.

After you install SDK you can use `gcloud` commands in your terminal. E.g. if you want to connect to your VM, you can use `gcloud beta compute ssh` command to connect to your VM. Go to the _VM Instances_ menu and start your VM. After the VM starts, select the dropdown menu next to the _SSH_ option in the list of instances and select the _View gcloud command_ option  [(Image reference)](../backstage/gcp_tutorial_images/ssh_command.png). You can copy the command it generates and use it in your console. [(Image reference)](../backstage/gcp_tutorial_images/login.png)

## Basic Setup

We now have a VM with Ubuntu 16.04 running. We want to run a Docker with TensorFlow image that is able to train models on GPU.

### Installing Nvidia Drivers

First, we will add a PPA package:

```
sudo apt update
sudo apt upgrade
sudo add-apt-repository ppa:graphics-drivers/ppa
```

You can check what is the current driver version number by running `sudo apt-cache search nvidia | grep driver`. Currently (Oct 2019), version number 430 is the newest:

```
sudo apt update 
sudo apt install nvidia-430
echo "sudo nvidia-smi -pm 1" >> ~/.bashrc
```

After you install the driver, reboot your VM:

```
sudo reboot

```

Finally after running the following command, you should see the status of your GPU  [(Image reference)](../backstage/gcp_tutorial_images/nvidia.png):

```
nvidia-smi
```

### Installing Docker

To install Docker simply follow the instruction in the [Install using the repository](https://docs.docker.com/install/linux/docker-ce/ubuntu/#install-using-the-repository) section of the Docker documentation. Then if we want to see the GPUs from inside the Docker container, we need to install `nvidia-docker` extension. Follow the Ubuntu installation procedure in [the readme](https://github.com/NVIDIA/nvidia-docker).

### Testing GPU

You should now be able to train with GPU in your Docker image, we can test it by downloading the official TensorFlow image and running a simple Python script:

```
sudo docker pull tensorflow/tensorflow:2.0.0-gpu-py3
sudo docker run --gpus all -it tensorflow/tensorflow:2.0.0-gpu-py3
python3
```

And then in python:

```
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
```

Running this code should return you a result like this:

```
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
```

